==== Anlage des Spark Context ====

Spark-Session erstellen und notwendige Frameworks importieren

In [ ]:
import findspark
findspark.init()
import pyspark
 

conf = pyspark.SparkConf()
#conf.set("spark.driver.bindAddress", "127.0.0.1")  # Localhost
#conf = conf.setMaster("local[*]")
conf = conf.setAppName("App")


sc = pyspark.SparkContext(conf=conf)

==== Vorverarbeitung der Datenquelle ===

==== Durchführen der Datenanlyse ====

Einlesen der Datei mit allen weltweiten Flugplätzen

In [ ]:
import csv
from io import StringIO

def analyze_airports():
    """
    Analysiert Flughäfen weltweit und gibt die Anzahl als String zurück.
    
    :return: Formatierten String mit den Ergebnissen.
    """

    # Daten einlesen und Header extrahieren
    airportRDD = sc.textFile("data/airports.csv")
    header = airportRDD.first()

    # CSV korrekt parsen, um Fehler durch Kommata innerhalb von Anführungszeichen zu vermeiden
    def parse_csv(line):
        reader = csv.reader(StringIO(line))
        return next(reader)

    # Filterkriterien anwenden und nur die relevanten Spalten speichern (1, 2, 3, 4, 5, 7)
    airportRDD = (
        airportRDD.filter(lambda line: line != header)  # Header entfernen
                  .map(parse_csv)  # CSV korrekt parsen
                  .filter(lambda cols: len(cols) > 7)  # Sicherstellen, dass genug Spalten vorhanden sind
                  .map(lambda cols: (cols[1].strip('"'),  # Name
                                    cols[2].strip('"'),  # Typ
                                    cols[3].strip('"'),  # Breitengrad
                                    cols[4].strip('"'),  # Längengrad
                                    cols[5].strip('"'),  # Höhe
                                    #cols[7].strip('"')
                                    ))  # Kontinent
                  .filter(lambda cols: cols[1] in ["small_airport", "medium_airport", "large_airport", "heliport"])  # Nur bestimmte Typen
    )

    # Anzahl der Flughäfen insgesamt
    total_airports = airportRDD.count()

    # Anzahl der Flughäfen nach Kategorien
    categorized_airports = (
        airportRDD.map(lambda cols: cols[1])  # Typ (Spalte 2)
                  .countByValue()
    )

    # Formatierte Ausgabe als String erstellen
    result = "Analyse weltweiter Flughäfen und Heliports\n"
    result += "=" * 50 + "\n"

    result += f"Die Anzahl der weltweiten Flughäfen und Heliports beträgt: {total_airports}\n"
    result += f"Davon Heliports: {categorized_airports.get('heliport', 0)}\n"
    result += f"Davon kleine Flughäfen: {categorized_airports.get('small_airport', 0)}\n"
    result += f"Davon mittelgroße Flughäfen: {categorized_airports.get('medium_airport', 0)}\n"
    result += f"Davon große Flughäfen: {categorized_airports.get('large_airport', 0)}\n"

    return result, airportRDD

# Beispielaufruf und Ausgabe
result, airportRDD = analyze_airports()
print(result)

# Optional: Die Daten aus airportRDD anzeigen lassen
airportRDD.collect()


Einlesen und filtern aller weltweit zugelassenen Flugobjekte, sortiert nach den Airlines mit den meisten Flugzeugen

In [ ]:
def analyze_aircraft_data():
    
    """
    Analysiert eine CSV-Datei mit Flugzeugdaten und gibt die Gesamtanzahl der registrierten Flugobjekte
    sowie die Top 20 Airlines nach der Anzahl der registrierten Flugzeuge zurück.

    :param file_path: Pfad zur CSV-Datei mit den Flugzeugdaten.
    :return: Ein Dictionary mit der Gesamtanzahl der Flugobjekte und den Top 20 Airlines.
    """

    # Daten einlesen und Header extrahieren
    aircraft_input_rdd = sc.textFile('data/aircraft-database-complete-2024-10.csv')
    header = aircraft_input_rdd.first()

    # Filterkriterien anwenden: Entfernen des Headers
    aircraft_raw_rdd = aircraft_input_rdd.filter(lambda line: line != header)

    # Anzahl der registrierten Flugobjekte
    Anzahl_registrierte_Flugobjekte = aircraft_raw_rdd.count()

    # RDD erstellen, um die Airlines nach Anzahl der registrierten Flugzeuge zu zählen
    aircraftRDD = (
        aircraft_raw_rdd.filter(lambda line: len(line.split(",")) > 16)
        .map(lambda line: line.split(",")[17].strip('"'))
        .filter(lambda airline: airline != "''")
        .map(lambda airline: (airline, 1))
        .reduceByKey(lambda a, b: a + b)
        .sortBy(lambda x: x[1], ascending=False)
    )
    
    # Top 20 Airlines nach Anzahl der registrierten Flugzeuge
    top_20_airlines = aircraftRDD.take(20)

    # Ergebnis als formatierten String zurückgeben
    result = f"Anzahl der weltweit gemeldeten, eindeutigen Flug- und Bodenobjekte: {Anzahl_registrierte_Flugobjekte}\n"
    result += "=" * 50 + "\n"
    result += "Top 20 Airlines nach Anzahl der registrierten Flugzeuge:\n"
    result += "=" * 50 + "\n"

    for airline, count in top_20_airlines:
        result += f"{airline}: {count}\n"

    return result, aircraft_raw_rdd, aircraftRDD

# Beispielaufruf und Ausgabe
result, aircraft_raw_rdd, aircraftRDD = analyze_aircraft_data()
print(result)


Auswertung der Bewegungsdaten mit visueller Darstellung 

In [ ]:
import folium
import time

# Analyse der Bewegungsdaten
def analyze_flight_movements(percentage=1.0):
    """
    Analysiert Bewegungsdaten von Flugzeugen und erstellt eine Karte des geografischen Bereichs.
    Misst dabei die Laufzeit für die Verarbeitung des Prozentsatzes der Daten.
    
    :param file_path: Pfad zur CSV-Datei mit den Bewegungsdaten
    :param percentage: Prozentsatz (zwischen 0 und 1) der Daten, die geladen werden sollen (Standard: 100%)
    :return: Die Ergebnisse der Auswertung, generierte Karte und das RDD der Daten sowie die benötigte Zeit
    """

    if not 0 < percentage <= 1:
        raise ValueError("Percentage muss zwischen 0 und 1 liegen")
    
    processing_start = time.time()  # Startzeit der Laufzeitmessung

    # Einlesen der Bewegungsdaten eines definierten Zeitraums in einem bestimmten geografischen Bereich
    myFileRDD = sc.textFile('data/processed_data.csv')
    header = myFileRDD.take(2)
    dataRDD = myFileRDD.filter(lambda line: line not in header)
    
    dataRDD = dataRDD.sample(False, percentage, seed=42)

    #Erfassen des Zeitraums in dem Daten die erhoben wurden - Skalierbarkeit gegeben aufgrund der Rückgabe eines Wertes
    start_time = dataRDD.map(lambda line: line.split(',')[0]).min()
    end_time = dataRDD.map(lambda line: line.split(',')[0]).max()

    #Geographische Analyse des betrachteten Luftraums - Skalierbarkeit gegeben aufgrund der Rückgabe und Visualisierung einzelner Werte

    # Längengrade sortieren und auslesen
    min_longitude = dataRDD.map(lambda line: float(line.split(',')[3])).min()
    max_longitude = dataRDD.map(lambda line: float(line.split(',')[3])).max()

    # Breitengrade (vermutlich Spalte mit Index 5 und 6)
    min_latitude = dataRDD.map(lambda line: float(line.split(',')[2])).min()
    max_latitude = dataRDD.map(lambda line: float(line.split(',')[2])).max()

    # Zentrum des Bereichs berechnen
    center_lat = (min_latitude + max_latitude) / 2
    center_lon = (min_longitude + max_longitude) / 2
    result_text = (
        f"Die erfassten Bewegungen von Flug- und Bodenobjekten wurden im Zeitraum vom "
        f"{start_time.replace('\"', '').replace('+00', '')} bis zum "
        f"{end_time.replace('\"', '').replace('+00', '')} (Zeitzone: UTC) ermittelt.\n"
        f"{'=' * 50}\n"
        "Geografische Abdeckung:\n"
        f"Längengrade: von {min_longitude:.4f}° bis {max_longitude:.4f}°\n"
        f"Breitengrade: von {min_latitude:.4f}° bis {max_latitude:.4f}°\n"
        "Die Daten wurden in folgendem Luftraum erhoben."
    )

    # Karte erstellen
    map = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Rechteck auf der Karte hinzufügen
    bounds = [[min_latitude, min_longitude], [max_latitude, max_longitude]]
    folium.Rectangle(
        bounds=bounds,
        color="red",
        weight=2,
        fill=True,
        fill_color="blue",
        fill_opacity=0.2,
        popup="Geographischer Bereich"
    ).add_to(map)

    processing_end = time.time()  # Endzeit der Laufzeitmessung
    elapsed_time = processing_end - processing_start  # Berechnete Laufzeit

    return result_text, map, dataRDD, elapsed_time 

result_text, map_result, dataRDD, elapsed_time = analyze_flight_movements()
print(result_text)
map_result

In [ ]:
# Analyse der Bewegungsdaten nach Flug- und Bodenobjekten

# Umstrukturierung der Daten zur Analyse der Daten nach Fahrzeug
def parse_line(line):
    parts = line.split(',')
    return (
        parts[1].strip(' "\t\r\n'),  # Key: Flugzeug-ID (icao24)
        {  # Value: Alle relevanten Daten als Dictionary
            "time": parts[0].strip('"'),
            "lat": float(parts[2].strip('"')),
            "lon": float(parts[3].strip('"')),
            "onground": int(parts[4].strip('"'))
        }
    )

# RDD transformieren: Gruppiere die Daten nach Flugzeug-ID - Skalierbarkeit gegeben aufgrund der Anlage eines neuen RDDs
flights_rdd = dataRDD.map(parse_line).groupByKey().mapValues(list)

#Anzahl der Flugzeuge, die innerhalb einer Stunde im betrachteten Bereich aktiv waren - Skalierbarkeit gegeben aufgrund der Rückgabe einzelner Werte
amount_of_aircrafts = flights_rdd.count()
print(f"Anzahl der im betrachteten Zeitraum aktiven Flug- und Bodenobjekte: {amount_of_aircrafts}")

#flights_rdd.take(1)

In [ ]:
# Kombination der Bewegungsdaten mit den Daten der registrierten Flug- und Bodenobjekten

def parse_type(line):
    parts = line.split(',')
    # Baseline-Verarbeitung ohne unnötige Komplexität
    return (
        parts[0].strip("'"),  # Flugzeug-ID (icao24)
        {
            "description": parts[5].strip("'") if len(parts) > 4 else "",
            "manufacturer": parts[13].strip("'") if len(parts) > 12 else "",
            "model": parts[14].strip("'") if len(parts) > 14 else "",
            "type": parts[15].strip("'") if len(parts) > 15 else "",
            "airline": parts[18].strip("'") if len(parts) > 16 else ""
        }
    )

# RDD bereinigen und nur relevante Daten aufnehmen
cleaned_type_rdd = aircraft_raw_rdd.map(parse_type)

# Kombination der Bewegunsdaten mit der Beschreibung der registrierten Flug- und Bodenobjekten
result_rdd = flights_rdd.leftOuterJoin(cleaned_type_rdd)

# Kombination der Daten in einem RDD
combined_rdd = result_rdd.map(lambda x: (
    x[0],  # Flugzeug-ID
    {
        "aircraft_info": x[1][1] if x[1][1] else {},  # Beschreibung, falls vorhanden
        "flight_data": sorted(x[1][0], key=lambda entry: entry["time"]) if x[1][0] else [] #Flugdaten sortiert nach Datum (lexografisch)
    }
))

#combined_rdd.take(1)

In [ ]:
#Analyse des kombinierten RDDs (Bewegungsdaten + Beschreibung)

#Ermittlung der Anzahl an Flug-Datensätze pro Flugzeug
countdatasetsRDD = combined_rdd.map(lambda x: (x[0],len(x[1]['flight_data']))).sortBy(lambda x: x[1], ascending=False)
first_10_elements = countdatasetsRDD.take(10)
print(f"Flugzeuge mit den meisten Datensätzen (Top 10) sind: {first_10_elements}  ")

print("")

# Bestimmung der Positon - Skalierbarkeit gegeben über Betrachtung einzelner Werte
airRDD = combined_rdd.map(lambda x: (x[0], x[1]['flight_data'][-1]["onground"]))
onground = airRDD.filter(lambda x: x[1]==1).count()
inair = airRDD.filter(lambda x: x[1]==0).count()
print(f"Status der Objekt zum letzten gemessenen Zeitpunkt: ")
print("Objekte aktuell in der Luft: " + str(onground))
print("Objekte aktuell am Boden: " + str(inair))


# Ermittlung der Position im Zeitraum - Skalierbarkeit gegeben über Betrachtung einzelner Werte
nextRDD = combined_rdd.map(lambda x: (x[0], sum(datapoint["onground"] for datapoint in x[1]['flight_data']), len(x[1]['flight_data'])))
calculatedRDD = nextRDD.map(lambda x: (x[0], x[1]/x[2]))

not_started = calculatedRDD.filter(lambda x: x[1] == 1).count()
print("")
print(f"Im gesamten betrachteten Zeitraum sind: ")
print(f"Objekte nicht am Boden geblieben: {not_started}")
not_landed = calculatedRDD.filter(lambda x: x[1] == 0).count()
print(f"Objekte nicht gelandet: {not_landed}")
startet_or_landed = calculatedRDD.filter(lambda x: x[1] < 1 and x[1] >0).count()
print(f"Objekte gestartet oder gelandet: {startet_or_landed}")


# Anzahl Starts und Anzahl Landungen

==== Visuelle Analysen ====

In [ ]:
import math

# Haversine-Funktion zum Berechnen der Distanz zwischen zwei Punkten
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Erd-Radius in Kilometern
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat / 2.0) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def find_nearest_airport(lat, lon, airports):
    """
    Findet den nächsten Flughafen zu gegebenen Koordinaten innerhalb von 10 km.
    Gibt den Flughafen mit der geringsten Distanz zurück.
    """
    nearest_airport = None
    min_distance = float("inf")

    for airport in airports:
        airport_id, _, airport_name, airport_lat, airport_lon = airport
        airport_lat = float(airport_lat)
        airport_lon = float(airport_lon)
        distance = haversine(lat, lon, airport_lat, airport_lon)
        if distance <= 10 and distance < min_distance:  # Innerhalb von 10 km und geringste Distanz
            min_distance = distance
            nearest_airport = (airport_id, airport_name)

    if nearest_airport:
        return nearest_airport
    return None, None

def extract_flights_with_airports(record, airports):
    """
    Extrahiert Flüge und fügt Flughafendaten hinzu.
    """
    icao24, combined_rdd = record
    aircraft_info = combined_rdd['aircraft_info']
    flight_data = combined_rdd['flight_data']

    # Filter: Ignoriere Datensätze mit "Surface Vehicle" in der Beschreibung
    if 'description' in aircraft_info and 'Surface Vehicle' in aircraft_info['description']:
        return []

    flights = []
    current_flight = []
    in_flight = False
    previous_entry = None  # Variable, um den vorherigen Datensatz zu speichern

    for entry in flight_data:
        # Überprüfung, ob es sich um einen Start handelt
        if not in_flight and entry['onground'] == 0 and previous_entry and previous_entry['onground'] == 1:
            in_flight = True
            current_flight = [entry]  # Startpunkt hinzufügen
        elif in_flight and entry['onground'] == 1:  # Landung erkannt
            current_flight.append(entry)  # Landepunkt hinzufügen
            # Flug-Informationen extrahieren
            start = current_flight[0]
            end = current_flight[-1]

            start_airport_id, start_airport_name = find_nearest_airport(start['lat'], start['lon'], airports)
            end_airport_id, end_airport_name = find_nearest_airport(end['lat'], end['lon'], airports)

            flight_info = {
                'start_time': start['time'],
                'start_coords': (start['lat'], start['lon']),
                'start_airport': {'id': start_airport_id, 'name': start_airport_name},
                'end_time': end['time'],
                'end_coords': (end['lat'], end['lon']),
                'end_airport': {'id': end_airport_id, 'name': end_airport_name},
            }
            flights.append((icao24, aircraft_info, flight_info))
            in_flight = False
            current_flight = []
        elif in_flight:  # Punkte während des Fluges sammeln
            current_flight.append(entry)

        # Speichere den aktuellen Eintrag als vorherigen für die nächste Iteration
        previous_entry = entry

    return flights

# Airports-Daten als Liste sammeln (Broadcast-Variable verwenden)
# Annahme: airportRDD ist das gegebene RDD mit den Flughafendaten
airports_list = airportRDD.collect()
broadcast_airports = sc.broadcast(airports_list)

# Anwenden der Extraktionsfunktion auf die RDD mit Flughafendaten
flights_rdd = combined_rdd.flatMap(lambda record: extract_flights_with_airports(record, broadcast_airports.value))

# Sammle alle Daten aus dem RDD
all_flights = flights_rdd.collect()

# Gib alle gesammelten Flugdaten aus
for flight in all_flights:
    print(flight)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import plotly.graph_objects as go

# Beispiel-Daten erweitern: Liste von Flügen
flights_data = flights_rdd.collect()

# Schritt 1: Flugverbindungen aggregieren (Start- und End-Airport-Paare zählen)
connections = defaultdict(int)
start_end_names = {}

for _, _, flight_info in flights_data:
    start_airport = flight_info['start_airport']
    end_airport = flight_info['end_airport']
    
    if start_airport['id'] and end_airport['id']:  # Nur valide Airports berücksichtigen
        start_id, start_name = start_airport['id'], start_airport['name']
        end_id, end_name = end_airport['id'], end_airport['name']
        
        connections[(start_id, end_id)] += 1
        start_end_names[start_id] = start_name
        start_end_names[end_id] = end_name

# Schritt 2: Netzwerkdiagramm erstellen
G = nx.Graph()

# Knoten und Kanten hinzufügen (nur Verbindungen mit mehr als 5 Flügen)
for (start, end), weight in connections.items():
    if weight > 5:  # Filter: Nur Verbindungen > 5
        G.add_edge(start, end, weight=weight)

# Schritt 3: Positionen berechnen
pos = nx.spring_layout(G, seed=42)

# Knoten und Kanten für Plotly vorbereiten
edges_x = []
edges_y = []
weights = []

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edges_x.extend([x0, x1, None])
    edges_y.extend([y0, y1, None])
    weights.append(edge[2]['weight'])

# Knoten vorbereiten
nodes_x = []
nodes_y = []
node_text = []

for node in G.nodes():
    x, y = pos[node]
    nodes_x.append(x)
    nodes_y.append(y)
    node_text.append(start_end_names.get(node, node))  # Namen verwenden

# Schritt 4: Interaktives Diagramm mit Plotly erstellen
fig = go.Figure()

# Kanten hinzufügen mit stärkerer Gewichtung
max_weight = max(weights) if weights else 1  # Maximale Gewichtung für Skalierung

for i, edge in enumerate(G.edges(data=True)):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        line=dict(width=(weights[i] / max_weight) * 100, color='black'),  # Stärkere Skalierung (100-fach)
        hoverinfo='none',
        mode='lines'))

# Knoten hinzufügen
fig.add_trace(go.Scatter(
    x=nodes_x, y=nodes_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    marker=dict(
        size=10,
        color='skyblue',
        line_width=2),
    hoverinfo='text'))

# Layout anpassen
fig.update_layout(
    title="Interaktives Netzwerkdiagramm der Flugverbindungen (Verbindungen > 5)",
    title_x=0.5,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=30),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor='white'
)

# Interaktives Diagramm anzeigen
fig.show()


Visualisierung der Bewegung und Beschreibung eines ausgewählten Flug- oder Bodenobjektes

In [ ]:
import folium
from IPython.display import display

def analyze_aircraft_route(selected_icao):
    # Filtere die Daten für die spezifische Flugzeug-ID im RDD
    filtered_data = combined_rdd.filter(lambda x: x[0] == selected_icao).collect()

    if filtered_data:
        # Hole die Daten für das ausgewählte Flugzeug
        icao24, route_data = filtered_data[0]
        aircraft_info = route_data["aircraft_info"]  # Zugriff auf die Flugzeugbeschreibung
        flight_data = route_data["flight_data"]  # Zugriff auf die Flugdaten

        print(f"Flugzeug-ID: {icao24}")

        # Erstelle die Karte, zentriert auf den ersten Datenpunkt
        first_lat = flight_data[0]["lat"]
        first_lon = flight_data[0]["lon"]
        map = folium.Map(location=[first_lat, first_lon], zoom_start=12)

        # Zeichne Linien zwischen aufeinanderfolgenden Punkten
        for i in range(len(flight_data) - 1):
            point_a = flight_data[i]
            point_b = flight_data[i + 1]

            # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
            if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
                folium.PolyLine(
                    locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                    color="blue",
                    weight=2.5,
                    opacity=0.8
                ).add_to(map)

        # Füge Marker nur für Landepunkte hinzu
        for point in flight_data:
            if point["onground"] == 1:  # Nur Landepunkte anzeigen
                folium.Marker(
                    location=(point["lat"], point["lon"]),
                    popup=f"Zeit: {point['time']}<br>Am Boden: {point['onground']}<br>Hersteller: {aircraft_info['manufacturer']}<br>Flugzeugtyp: {aircraft_info['model']}<br>Airline: {aircraft_info['airline']}",
                    icon=folium.Icon(color="green", icon="info-sign")
                ).add_to(map)

        # Rückgabe der Karte
        return map

    else:
        return f"Keine Daten für Flugzeug-ID '{selected_icao}' gefunden."


result = analyze_aircraft_route("c04fdd")
display(result)

Visualisierung der Bewegung und Beschreibung mehrerer, zufällig ausgewählter Flug- oder Bodenobjekte

In [ ]:
import folium
from IPython.display import display
import random

# Zufällige Farben für die Flugzeuge
def get_random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

def analyze_aircraft_routes(number_of_samples):
    # Nimm eine beliebige Anzahl zufälliger Flugzeuge aus dem RDD
    random_aircrafts = combined_rdd.takeSample(withReplacement=False, num=number_of_samples)

    # Erstelle eine Karte, zentriert auf einen Beispielpunkt
    if random_aircrafts:
        # Hole den ersten Punkt zur Zentrierung der Karte
        first_lat = random_aircrafts[0][1]["flight_data"][0]["lat"]
        first_lon = random_aircrafts[0][1]["flight_data"][0]["lon"]
        map = folium.Map(location=[first_lat, first_lon], zoom_start=6)

        # Iteriere über die zufälligen Flugzeuge
        for icao24, data in random_aircrafts:
            aircraft_info = data["aircraft_info"]  # Infos über das Flugzeug
            flight_data = data["flight_data"]      # Flugdaten

            # Sichere Abfrage von Typ, Modell und Hersteller mit Standardwerten
            aircraft_type = aircraft_info.get("type", "Unknown")
            aircraft_model = aircraft_info.get("model", "Unknown")
            aircraft_manufacturer = aircraft_info.get("manufacturer", "Unknown")
            aircraft_airline = aircraft_info.get("airline", "Unknown")

            # Generiere eine zufällige Farbe für die Linien
            color = get_random_color()

            # Zeichne Linien zwischen aufeinanderfolgenden Punkten
            for i in range(len(flight_data) - 1):
                point_a = flight_data[i]
                point_b = flight_data[i + 1]

                # Zeichne nur Linien, wenn beide Punkte gültige Koordinaten haben
                if point_a["lat"] and point_a["lon"] and point_b["lat"] and point_b["lon"]:
                    folium.PolyLine(
                        locations=[(point_a["lat"], point_a["lon"]), (point_b["lat"], point_b["lon"])],
                        color=color,
                        weight=2.5,
                        opacity=0.8
                    ).add_to(map)

            # Füge Marker nur für Landepunkte hinzu
            for point in flight_data:
                if point["onground"] == 1:  # Nur Landepunkte anzeigen
                    folium.Marker(
                        location=(point["lat"], point["lon"]),
                        popup=(
                            f"Flugzeug: {icao24}<br>"
                            f"Hersteller: {aircraft_manufacturer}<br>"
                            f"Modell: {aircraft_model}<br>"
                            f"Airline: {aircraft_airline}<br>"
                            f"Zeit: {point['time']}<br>"
                            f"Am Boden: {point['onground']}"
                        ),
                        icon=folium.Icon(color="green", icon="info-sign")
                    ).add_to(map)

        # Zeige die Karte im Notebook
        return map
    else:
        return "Keine Daten gefunden."
    
result = analyze_aircraft_routes(20)
display(result)
    

Heatmap zur visuellen Auswertung aller Bewegungsdaten

In [ ]:
import folium
from folium.plugins import HeatMap

# Map und Filter: Direktes Überprüfen der Konvertierbarkeit in float
# 1 Dezimalstelle: ~11,1 km Genauigkeit eine Nachkommastelle reicht für globale Karten oder grobe Cluster.
# 2 Dezimalstellen: ~1,1 km Genauigkeit gut für Städte oder Regionen.
# 3 Dezimalstellen: ~110 m Genauigkeit ideal für Stadtviertel oder grobe Stadtanalysen.
# Ab 3 Nachkommastellen geht nix mehr

def extract_coordinates(line):
    try:
        parts = line.split(',')
        return round(float(parts[3]), 2), round(float(parts[4]), 2)  # Breitengrad, Längengrad
    except (ValueError, IndexError):
        return None  # Falls Konvertierung oder Zugriff fehlschlägt

location_rdd = dataRDD.map(extract_coordinates).filter(lambda x: x is not None)

number = location_rdd.count()
print(f"Anzahl der Daten vor der Reduktion {number}")


# Map-Reduce: Zähle, wie oft jede Kombination von Breitengrad und Längengrad vorkommt
coordinate_counts = location_rdd.map(lambda coord: (coord, 1)).reduceByKey(lambda a, b: a + b)

# Ergebnisse anzeigen
print(coordinate_counts.take(5))
print(f"Anzahl Koordinaten nach der Reduktion {coordinate_counts.count()}")

# Beispiel: Aggregierte Koordinaten mit Häufigkeiten (ersetzt durch deine Daten)
aggregated_data = coordinate_counts.collect()

# Vorbereitung der Heatmap-Daten
heatmap_data = [(lat_lon[0], lat_lon[1], count) for lat_lon, count in aggregated_data]

# Erstelle die Karte
if heatmap_data:
    # Zentriere die Karte auf den ersten Punkt und passe die Zoom-Stufe an
    map = folium.Map(location=[heatmap_data[0][0], heatmap_data[0][1]], zoom_start=6)

    # HeatMap hinzufügen (Radius und max_zoom können angepasst werden)
    HeatMap(heatmap_data, radius=15, max_zoom=13).add_to(map)

    # Karte anzeigen
    display(map)
else:
    print("Keine gültigen Punkte für die Heatmap gefunden.")

==== Untersuchung der Analysen ====

Definition der Analyse-Funktionen

In [50]:
import time
import psutil
import datetime

def get_system_info():
    # CPU-Informationen
    cpu_count = psutil.cpu_count(logical=False)  # Anzahl physikalischer CPU-Kerne
    logical_cpu_count = psutil.cpu_count(logical=True)  # Anzahl logischer CPUs (mit Hyper-Threading)
    cpu_freq = psutil.cpu_freq()  # CPU Frequenz
    cpu_percent = psutil.cpu_percent(interval=1)  # CPU-Auslastung in Prozent

    # RAM-Informationen
    virtual_memory = psutil.virtual_memory()  # Virtueller Speicher (RAM)
    total_memory = virtual_memory.total  # Gesamtgröße des RAM
    available_memory = virtual_memory.available  # Verfügbarer RAM
    used_memory = virtual_memory.used  # Genutzter RAM
    memory_percent = virtual_memory.percent  # Prozentsatz des verwendeten RAM


    # Ausgabe der gesammelten Informationen
    result = "=" * 50
    result += "System Informationsübersicht"
    result += "=" * 50\

    result += "\nCPU Infos:\n"
    result += f"  - Anzahl der physischen CPU-Kerne: {cpu_count}\n"
    result += f"  - Anzahl der logischen CPU-Kerne (inkl. Hyper-Threading): {logical_cpu_count}\n"
    result += f"  - Aktuelle CPU Frequenz: {cpu_freq.current} MHz\n"
    result += f"  - Aktuelle CPU Auslastung: {cpu_percent}%\n"

    result += "\nRAM Infos:\n"
    result += f"  - Gesamter RAM: {total_memory / (1024 ** 3):.2f} GB\n"
    result += f"  - Verfügbarer RAM: {available_memory / (1024 ** 3):.2f} GB\n"
    result += f"  - Genutzter RAM: {used_memory / (1024 ** 3):.2f} GB\n"
    result += f"  - RAM Auslastung: {memory_percent}%"

    return result



# Datenverteilung im CLuster
def analyze_partition_distribution(rdd):
    """
    Analysiert die Verteilung der Daten in einem RDD über die Partitionen.

    :param rdd: Das RDD, das analysiert werden soll.
    """
    # Funktion, um die Größe jeder Partition zu berechnen
    def partition_sizes(index, iterator):
        yield index, sum(1 for _ in iterator)

   # Daten je Partition sammeln
    partition_info = rdd.mapPartitionsWithIndex(partition_sizes).collect()

    # Anzahl der Partitionen ermitteln
    num_partitions = rdd.getNumPartitions()

    #Ergebnisse formatieren
    number_data = f"Anzahl der Datensätze: {rdd.count()}\n"
    number_partition = f"Anzahl der Partitionen: {num_partitions}\n"
    result = "Datenverteilung auf Partitionen:\n"

    # Ausgabe der Verteilung pro Partition
    for partition, size in partition_info:
        result += f"Partition {partition}: {size} Datensätze\n"

    return number_data, number_partition, result

# Fehlertoleranz

# Laufzeit
import time
def measure_transformation_time(func, *args, **kwargs):
    """
    Misst die Laufzeit einer Funktion und gibt das Ergebnis sowie die Laufzeit zurück.
    
    :param func: Die Funktion, deren Laufzeit gemessen werden soll
    :param args: Argumente der Funktion
    :param kwargs: Keyword-Argumente der Funktion
    :return: Das Ergebnis der Funktion (als Tupel) und die Laufzeit in Sekunden
    """
    start_time = time.time()
    
    # Führe die Funktion aus und erhalte die Rückgabe
    result = func(*args, **kwargs)
    
    end_time = time.time()
    duration = end_time - start_time
    
    # Zeige die Laufzeit an
    print(f"Die Funktion '{func.__name__}' hat {duration:.4f} Sekunden gedauert.")
    
    # Rückgabe: Das Ergebnis als Tupel und die Laufzeit
    return result, duration


Hardwareanalyse

In [ ]:
# Hardwareanalyse
print(get_system_info())

Analyse 1 - Europäische Flughäfen

In [ ]:
#Laufzeit
measure_transformation_time(analyze_airports)

#Datenverteilung
number_data, number_partition, result = analyze_partition_distribution(airportRDD)
print(number_data + number_partition + "\n" + result)

Analyse 2 - Flugzeugbeschreibungen

In [ ]:
#Laufzeit
measure_transformation_time(analyze_aircraft_data)

#Datenverteilung
number_data, number_partition, result = analyze_partition_distribution(aircraftRDD)
print(number_data + number_partition + "\n" + result)

Analyse 3 - Bewegungsdaten

In [ ]:
#Ausführung, Datenverteilung, Laufzeit
result_text, map_result, dataRDD, elapsed_time = analyze_flight_movements(percentage=1.0)
print(f"Laufzeit:  {elapsed_time}")
print(result_text)
print()
number_data, number_partition, result = analyze_partition_distribution(dataRDD)
print(number_data + number_partition)


#Skalierbarkeit, Datenverteilung und Laufzeiten
percentages = [0.1, 0.2, 0.5, 1.0]  # 10%, 20%, 50%, 100%

for percentage in percentages:
    print(f"\nVerarbeite {percentage*100}% der Daten...")
    result_text, map_result, dataRDD, elapsed_time = analyze_flight_movements(percentage=percentage)
    print(f"Verarbeitungszeit für {percentage*100}% der Daten: {elapsed_time:.2f} Sekunden")
    number_data, number_partition, result = analyze_partition_distribution(dataRDD)
    print(number_data + number_partition)
    #print(result_text)
    map_result  # Karte wird angezeigt

#Fehlertoleranz


==== Test Parquet ====

In [58]:
#from pyspark.sql import SparkSession

#spark = SparkSession.builder \
#    .appName("Inspect Parquet") \
#    .getOrCreate()

#df = spark.read.parquet("data/bigDataSet_v2.parquet")
#df.show(2)  # Zeigt die ersten 5 Zeilen der Parquet-Datei

#df.printSchema()
#print("Spaltennamen:", df.columns)

# Umwandeln des DataFrames in ein RDD
#rdd = df.rdd

# Bereinigen der Row-Daten und Zugreifen auf die richtigen Felder
#rdd_values = rdd.map(lambda row: (row[0].strip(), row[1], row[2], row[3]))

# Ausgabe der ersten 3 Zeilen
#print(rdd_values.take(3))